
# Balanceamento de Carteira de Investimentos 📈📊

Notebook para análise e balanceamento de carteiras de ativos financeiros.  
Inclui cálculo de pesos ótimos, redistribuição de ativos e avaliação de métricas de risco.

---


In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive, files
import os
import yfinance as yf
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/')
dados = pd.read_csv("tickers_final.csv", decimal=',')


Mounted at /content/drive


In [ ]:
# número de linhas e colunas
dados.head(3)

,ID,Ticker,Empresa,Yahoo Finance Ticker,Google Finance Ticker
0,0.0,A1MD34,Allied Tecnologia,A1MD34,A1MD34
1,1.0,BTLT39,Brookfield Infra Partners,BTLT39.SA,BTLT39
2,2.0,CCRO3,CCR,CCRO3.SA,CCRO3


In [ ]:
# Alterando diretamente a coluna 'Yahoo Finance Ticker' com base nas condições específicas
dados.loc[dados['Yahoo Finance Ticker'] == 'A1MD34', 'Yahoo Finance Ticker'] = 'AMD'
dados.loc[dados['Yahoo Finance Ticker'] == 'BTLT39', 'Yahoo Finance Ticker'] = 'TLT'
dados.loc[dados['Yahoo Finance Ticker'] == 'CORN11', 'Yahoo Finance Ticker'] = 'CORN'
dados.loc[dados['Yahoo Finance Ticker'] == 'M2ST34', 'Yahoo Finance Ticker'] = 'MSTR'
dados.loc[dados['Yahoo Finance Ticker'] == 'XINA11', 'Yahoo Finance Ticker'] = 'FXI'
dados.loc[dados['Yahoo Finance Ticker'] == 'ROXO34', 'Yahoo Finance Ticker'] = 'NU'
dados.loc[dados['Yahoo Finance Ticker'] == 'SNEC34', 'Yahoo Finance Ticker'] = 'SONY'
dados.loc[dados['Yahoo Finance Ticker'] == 'MELI34', 'Yahoo Finance Ticker'] = 'MELI'
dados.loc[dados['Yahoo Finance Ticker'] == 'TSLA34', 'Yahoo Finance Ticker'] = 'TSLA'

# Exibindo as primeiras 3 linhas do DataFrame para verificar as alterações
print(dados.head(3))

# Salvando o DataFrame atualizado
dados.to_csv("tickers_final.csv", index=False)
print("Arquivo atualizado salvo como 'tickers_final.csv' no Google Drive.")

    ID  Ticker                    Empresa Yahoo Finance Ticker  \
0  0.0  A1MD34          Allied Tecnologia                  AMD   
1  1.0  BTLT39  Brookfield Infra Partners                  TLT   
2  2.0   CCRO3                        CCR             CCRO3.SA   

  Google Finance Ticker  
0                A1MD34  
1                BTLT39  
2                 CCRO3  
Arquivo atualizado salvo como 'tickers_final.csv' no Google Drive.


In [ ]:
# Carregando o DataFrame com os tickers
df = pd.read_csv('tickers_final.csv')

# Períodos desejados em dias
periods = [30, 60, 90, 120, 150]

# Função para calcular preços de fechamento, retornos médios e desvios padrão
def calculate_metrics(ticker, periods):
    try:
        # Obtendo dados históricos para 6 meses
        data = yf.Ticker(ticker).history(period='6mo')
        close_prices = data['Close']  # Preços de fechamento

        # Inicializando dicionários para armazenar métricas
        avg_returns = {}
        std_devs = {}

        for period in periods:
            period_returns = close_prices.pct_change().tail(period)  # Retornos diários
            avg_returns[f'Avg Return {period}d'] = period_returns.mean()
            std_devs[f'Std Dev {period}d'] = period_returns.std()

        return avg_returns, std_devs
    except Exception as e:
        print(f"Erro ao processar {ticker}: {e}")
        return {f'Avg Return {p}d': None for p in periods}, {f'Std Dev {p}d': None for p in periods}

# Calculando métricas para todos os tickers
for ticker in df['Yahoo Finance Ticker']:
    avg_returns, std_devs = calculate_metrics(ticker, periods)

    # Adicionando as métricas calculadas ao DataFrame
    for col, value in avg_returns.items():
        df.loc[df['Yahoo Finance Ticker'] == ticker, col] = value
    for col, value in std_devs.items():
        df.loc[df['Yahoo Finance Ticker'] == ticker, col] = value

# Exibindo e salvando o DataFrame atualizado
print(df)

ERROR:yfinance:$IBOV: possibly delisted; no price data found  (period=6mo) (Yahoo error = "No data found, symbol may be delisted")


      ID  Ticker                    Empresa Yahoo Finance Ticker  \
0   0.00  A1MD34          Allied Tecnologia                  AMD   
1   1.00  BTLT39  Brookfield Infra Partners                  TLT   
2   2.00   CCRO3                        CCR             CCRO3.SA   
3   3.00  CORN11           iShares Corn ETF                 CORN   
4   4.00   EMBR3                    Embraer             EMBR3.SA   
5   5.00   FIQE3                FIQ Energia             FIQE3.SA   
6   6.00   INTB3                  Intelbras             INTB3.SA   
7   7.00  M2ST34               Macro Usinas                 MSTR   
8   8.00   MGLU3             Magazine Luiza             MGLU3.SA   
9   9.00   PNVL3                     Panvel             PNVL3.SA   
10 10.00   POMO4                  Marcopolo             POMO4.SA   
11 11.00   RANI3    IRANI Papel e Embalagem             RANI3.SA   
12 12.00   RENT3        Localiza Rent a Car             RENT3.SA   
13 13.00  ROXO34                   Roku Inc     

**CALCULO DE RETORNOS E DESVIOS PADRÕES NO PERIODO X**

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

# Definindo a data inicial e final da análise
start_date = '2023-05-01'  # Ajuste conforme necessário
end_date = '2024-11-07'     # Ajuste conforme necessário

# Carregar o DataFrame com os tickers
df = pd.read_csv("tickers_final.csv")

# Criar colunas para armazenar os resultados
periods = [30, 60, 90, 120]  # Períodos de interesse
for period in periods:
    df[f'Avg Return {period}d'] = None
    df[f'Std Dev Return {period}d'] = None  # Adicionando colunas para o desvio padrão

# Criar DataFrame para armazenar preços ajustados diários de todos os ativos
daily_prices_df = pd.DataFrame()

# Função para obter a cotação do dólar (USD/BRL)
def get_usd_to_brl(start, end):
    usd_brl_data = yf.Ticker('BRL=X').history(start=start, end=end)
    return usd_brl_data['Close']

# Função para processar cada ativo
def process_ticker(ticker, usd_brl_rate):
    try:
        # 1. Obter dados históricos entre as datas especificadas
        data = yf.Ticker(ticker).history(start=start_date, end=end_date)

        # 2. Filtrar apenas dias úteis
        data = data[data.index.dayofweek < 5]  # Remove finais de semana

        # 3. Ajustar preços para ativos cotados em dólar
        dollar_tickers = ['MELI', 'A1MD34', 'BTLT39', 'NU', 'XINA11', 'M2ST34']
        if ticker in dollar_tickers:  # Adicione aqui todos os tickers cotados em dólar
            data['Close'] = data['Close'] * usd_brl_rate.reindex(data.index, method='ffill')  # Ajusta o preço com a cotação do dólar

        # 4. Armazenar preços ajustados diários
        global daily_prices_df
        daily_prices_df[ticker] = data['Close']  # Adiciona a coluna do ativo ao DataFrame de preços diários

        # 5. Calcular retornos tradicionais e desvio padrão para períodos específicos
        avg_returns = {}
        for period in periods:
            if len(data) >= period:  # Verifica se há dados suficientes
                initial_price = data['Close'].iloc[-period]  # Preço inicial
                final_price = data['Close'].iloc[-1]         # Preço final
                avg_return = ((final_price - initial_price) / initial_price) * 10000  # Retorno em porcentagem
                avg_returns[f'Avg Return {period}d'] = avg_return

                # Cálculo do desvio padrão
                daily_returns = data['Close'].pct_change().dropna()  # Calcula os retornos diários
                std_dev = daily_returns.rolling(window=period).std().iloc[-1]  # Desvio padrão para o período
                avg_returns[f'Std Dev Return {period}d'] = std_dev * 10000  # Multiplicando por 10.000

        return avg_returns
    except Exception as e:
        print(f"Erro ao processar o ticker {ticker}: {e}")
        return {f'Avg Return {p}d': None for p in periods}

# Obter a cotação do dólar (USD/BRL) para o período de interesse
usd_brl_rate = get_usd_to_brl(start_date, end_date)

# Processar cada ativo
for index, row in df.iterrows():
    ticker = row['Yahoo Finance Ticker']
    avg_returns = process_ticker(ticker, usd_brl_rate)

    # Adicionar os resultados ao DataFrame principal
    for period, value in avg_returns.items():
        df.at[index, period] = value

# Exibir os resultados
print(df[['Yahoo Finance Ticker'] + [f'Avg Return {p}d' for p in periods] + [f'Std Dev Return {p}d' for p in periods]])

# Salvar o DataFrame principal atualizado
df.to_csv("tickers_final.csv", index=False)  # Salvar os resultados principais
print("Resultados salvos em 'tickers_final.csv'.")

# Salvar os preços ajustados diários em um novo arquivo CSV
# daily_prices_df.to_csv("daily_prices.csv", index=True)  # Index=True para incluir as datas como índice
# print("Preços ajustados diários salvos em 'daily_prices.csv'.")

# Salvar os preços ajustados diários em um novo arquivo CSV
daily_prices_df.to_excel("daily_prices.xlsx", index=True)  # Index=True para incluir as datas como índice
print("Preços ajustados diários salvos em 'daily_prices.xlsx'.")




FileNotFoundError: [Errno 2] No such file or directory: 'tickers_final.csv'

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Carregar os preços diários ajustados
daily_prices = pd.read_csv("daily_prices.csv", index_col=0, parse_dates=True)

# Calcular os retornos diários para os últimos 30 dias
returns_daily_30d = daily_prices.tail(30).pct_change().dropna()

# Calcular a matriz de correlação entre os retornos diários
correlation_matrix_30d = returns_daily_30d.corr()

# Plotar a matriz de correlação como mapa de calor
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix_30d,
            annot=True,
            cmap='coolwarm',
            fmt=".2f",
            cbar=True,
            xticklabels=correlation_matrix_30d.columns,
            yticklabels=correlation_matrix_30d.columns)
plt.title("Correlation Matrix (Daily Returns - 30 Days)", fontsize=16)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()
